In [2]:
pip install imblearn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for imbalanced-learn from https://files.pythonhosted.org/packages/a3/9e/fbe60a768502af54563dcb59ca7856f5a8833b3ad5ada658922e1ab09b7f/imbalanced_learn-0.11.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/235.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/235.6 kB ? eta -:--:--
   ----- --------------------------------- 30.7/235.6 kB 435.7 kB/s eta 0:00:01
   ---------- ---------------------------- 61.4/235.6 kB 544.7 kB/s eta 0:00:01
   --------------------------------- ------ 194.6/235.6 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 235.6/235.6 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
from catboost import cv, Pool
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.metrics import AUC
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
RS = 121212
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTE

C:\Users\User\anaconda3\envs\new\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\User\anaconda3\envs\new\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\User\anaconda3\envs\new\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


ModuleNotFoundError: No module named 'imblearn'

# Загрузка данных

In [ ]:
df = pd.read_csv('train_dataset_Самолет.csv', low_memory=False, parse_dates=['report_date'])

In [ ]:
df.head()

plt.figure(figsize=(15, 10))
sns.heatmap(df.select_dtypes([int, float]).corr())
plt.show()

In [ ]:
data = df.dropna(thresh=int(len(df)*0.95), axis=1).copy()
data.drop(columns=['col1454'], inplace=True)

In [ ]:
data = data.fillna(data.mean())

In [ ]:
data.head()

In [ ]:
data_f = data.copy()

plt.figure(figsize=(15, 10))
sns.heatmap(data.corr())
plt.show()

In [ ]:
client_id = data['client_id']

In [ ]:
data.drop(columns='client_id', inplace=True)

# Модели

## Создание baseline модели

In [ ]:
features = data.drop(columns=['target', 'report_date'])

In [ ]:
target = data['target']

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, stratify=target,
                                                                            random_state=RS)

In [ ]:
features_train, target_train = SMOTE().fit_resample(features_train, target_train)

In [ ]:
params = {
    'loss_function': 'Logloss',
    'iterations': 300,
    'custom_loss': ['AUC', 'F1'],
    'random_seed': RS,
    'learning_rate': 0.2
}

In [ ]:
cv_data = cv(
    params=params,
    pool=Pool(features_train, label=target_train),
    fold_count=5, # Разбивка выборки на 5 кусочков
    shuffle=True, # Перемешаем наши данные
    partition_random_seed=RS,
    plot=True,
    stratified=True, 
    verbose=False
)

In [ ]:
cv_data

In [ ]:
baseline_auc_valid = cv_data['test-AUC-mean'].tail(1)

In [ ]:
baseline_f1_valid = cv_data['test-F1-mean'].tail(1)

In [ ]:
baseline_auc_valid

In [ ]:
baseline_f1_valid

### Валидация модели

In [ ]:
model = CatBoostClassifier(loss_function='Logloss',
    iterations=300,
    custom_loss=['AUC', 'F1'],
    random_seed= RS,
    learning_rate= 0.2,
    verbose=False)

In [ ]:
model.fit(features_train, target_train)

In [ ]:
pred = model.predict(features_test)

In [ ]:
baseline_auc_test = roc_auc_score(target_test, pred)

In [ ]:
baseline_auc_test

In [ ]:
baseline_f1_test = f1_score(target_test, pred)

In [ ]:
baseline_f1_test

In [ ]:
cm = confusion_matrix(target_test, pred, normalize='true')
ConfusionMatrixDisplay(confusion_matrix=cm).plot()
plt.show()

In [ ]:

# Get predicted probabilities for each class
preds_proba = model.predict_proba(features_test)
# Get predicted RawFormulaVal


In [ ]:
preds_proba

In [ ]:
df = pd.DataFrame(preds_proba)

In [ ]:
(df[1] > 0.5).sum()

In [ ]:
preds = model.predict(features_test)

In [ ]:
preds

In [ ]:
from math import exp

In [ ]:
exp( preds_proba) / (1+ exp( preds_proba))

In [ ]:
test['score'] = model.predict_proba(test[model_cols])
sample_submission = test.loc[:, ['report_date', 'client_id', 'score']]

## Модель, обученная на клиентах, которые имеют таргет 0 и 1

In [ ]:
train, test = train_test_split(data_f.drop(columns='report_date'), test_size=0.3, random_state=RS, stratify=data['target'])

In [ ]:
one_id = train.query('target == 1')['client_id']

In [ ]:
clients_changed = train.query('client_id in @one_id')

In [ ]:
clients_changed.head()

In [ ]:
features_train = clients_changed.drop(columns=['target', 'client_id'])

In [ ]:
target_train = clients_changed['target']

In [ ]:
features_train, target_train = SMOTE().fit_resample(features_train, target_train)

In [ ]:
params = {
    'loss_function': 'Logloss',
    'iterations': 300,
    'custom_loss': ['AUC', 'F1'],
    'random_seed': RS,
    'learning_rate': 0.2
}

In [ ]:
cv_data = cv(
    params=params,
    pool=Pool(features_train, label=target_train),
    fold_count=5, # Разбивка выборки на 5 кусочков
    shuffle=True, # Перемешаем наши данные
    partition_random_seed=RS,
    plot=True,
    stratified=True, 
    verbose=False
)

In [ ]:
cv_data

In [ ]:
changed_auc_valid = cv_data['test-AUC-mean'].tail(1)

In [ ]:
changed_f1_valid = cv_data['test-F1-mean'].tail(1)

In [ ]:
changed_auc_valid

In [ ]:
changed_f1_valid

### Валидация модели

In [ ]:
model = CatBoostClassifier(loss_function='Logloss',
    iterations=300,
    custom_loss=['AUC', 'F1'],
    random_seed= RS,
    learning_rate= 0.2,
    verbose=False)

In [ ]:
model.fit(features_train, target_train)

In [ ]:
features_test = test.drop(columns=['target', 'client_id'])

In [ ]:
target_test = test['target']

In [ ]:
pred = model.predict(features_test)

In [ ]:
changed_auc_test = roc_auc_score(target_test, pred)

In [ ]:
changed_auc_test

In [ ]:
changed_f1_test = f1_score(target_test, pred)

In [ ]:
changed_f1_test

In [ ]:
cm = confusion_matrix(target_test, pred, normalize='true')
ConfusionMatrixDisplay(confusion_matrix=cm).plot()
plt.show()

Делаем вывод о том, что модель не обучается на таких данных

## Модель, обученная на измененных данных - добавили наблюдения с таргетом 1 для получения трех единиц по уникальному пользователю, совершившему покупку

In [ ]:
train, test = train_test_split(data_f.drop(columns='report_date'), test_size=0.3, random_state=RS, stratify=data['target'])

In [ ]:
clients = train.query('target == 1').groupby('client_id').agg('count').reset_index()

In [ ]:
add_two = clients.query('target == 1')['client_id']

In [ ]:
add_two

In [ ]:
add_one = clients.query('target == 2')['client_id']

In [ ]:
add_two_values = train.query('client_id in @add_two & target==1').groupby('client_id').agg('mean').reset_index()

In [ ]:
add_one_values = train.query('client_id in @add_one & target==1')

In [ ]:
data_added = pd.concat([train, add_two_values, add_two_values, add_one_values], axis=0)

In [ ]:
data_added

In [ ]:
features_train = data_added.drop(columns=['target', 'client_id'])

In [ ]:
target_train = data_added['target']

In [ ]:
features_train, target_train = SMOTE().fit_resample(features_train, target_train)

In [ ]:
params = {
    'loss_function': 'Logloss',
    'iterations': 300,
    'custom_loss': ['AUC', 'F1'],
    'random_seed': RS,
    'learning_rate': 0.2
}

In [ ]:
cv_data = cv(
    params=params,
    pool=Pool(features_train, label=target_train),
    fold_count=5, # Разбивка выборки на 5 кусочков
    shuffle=True, # Перемешаем наши данные
    partition_random_seed=RS,
    plot=True,
    stratified=True, 
    verbose=False
)

In [ ]:
cv_data

In [ ]:
added_auc_valid = cv_data['test-AUC-mean'].tail(1)

In [ ]:
added_f1_valid = cv_data['test-F1-mean'].tail(1)

In [ ]:
added_auc_valid

In [ ]:
added_f1_valid

### Валидация модели

In [ ]:
model = CatBoostClassifier(loss_function='Logloss',
    iterations=300,
    custom_loss=['AUC', 'F1'],
    random_seed= RS,
    learning_rate= 0.2,
    verbose=False)

In [ ]:
model.fit(features_train, target_train)

In [ ]:
features_test = test.drop(columns=['target', 'client_id'])

In [ ]:
target_test = test['target']

In [ ]:
pred = model.predict(features_test)

In [ ]:
added_auc_test = roc_auc_score(target_test, pred)

In [ ]:
added_auc_test

In [ ]:
added_f1_test = f1_score(target_test, pred)

In [ ]:
added_f1_test

In [ ]:
cm = confusion_matrix(target_test, pred, normalize='true')
ConfusionMatrixDisplay(confusion_matrix=cm).plot()
plt.show()

## Модель, обученная на измененных данных - заполнили единицами таргет у всех уникальных пользователей, у кого встречается хотя бы одна единица в таргете

In [ ]:
train, test = train_test_split(data_f.drop(columns='report_date'), test_size=0.3, random_state=RS, stratify=data_f['target'])

In [ ]:
train.query('target == 1')

In [ ]:
one_id = train.query('target == 1')['client_id']

In [ ]:
one_id

In [ ]:
criteria = 'client_id in @one_id'

In [ ]:
train.loc[train.eval(criteria), 'target'] = 1

In [ ]:
train.query('client_id in @one_id').loc[:, 'target']

In [ ]:
features_train = train.drop(columns=['target', 'client_id'])

In [ ]:
target_train = train['target']

In [ ]:
features_train, target_train = SMOTE().fit_resample(features_train, target_train)

In [ ]:
params = {
    'loss_function': 'Logloss',
    'iterations': 300,
    'custom_loss': ['AUC', 'F1'],
    'random_seed': RS,
    'learning_rate': 0.2
}

In [ ]:
cv_data = cv(
    params=params,
    pool=Pool(features_train, label=target_train),
    fold_count=5, # Разбивка выборки на 5 кусочков
    shuffle=True, # Перемешаем наши данные
    partition_random_seed=RS,
    plot=True,
    stratified=True, 
    verbose=False
)

In [ ]:
cv_data

In [ ]:
filled_auc_valid = cv_data['test-AUC-mean'].tail(1)

In [ ]:
filled_f1_valid = cv_data['test-F1-mean'].tail(1)

In [ ]:
filled_auc_valid

In [ ]:
filled_f1_valid

### Валидация модели

In [ ]:
model = CatBoostClassifier(loss_function='Logloss',
    iterations=300,
    custom_loss=['AUC', 'F1'],
    random_seed= RS,
    learning_rate= 0.2,
    verbose=False)

In [ ]:
model.fit(features_train, target_train)

In [ ]:
features_test = test.drop(columns=['target', 'client_id'])

In [ ]:
target_test = test['target']

In [ ]:
pred = model.predict(features_test)

In [ ]:
filled_auc_test = roc_auc_score(target_test, pred)

In [ ]:
filled_auc_test

In [ ]:
filled_f1_test = f1_score(target_test, pred)

In [ ]:
filled_f1_test

In [ ]:
cm = confusion_matrix(target_test, pred, normalize='true')
ConfusionMatrixDisplay(confusion_matrix=cm).plot()
plt.show()

## Бейслайн модель, обученная на данных с новыми фичами

In [ ]:
train, test = train_test_split(data_f, test_size=0.3, random_state=RS, stratify=data['target'])

In [ ]:
def date_maker(row):
    if row['report_date'].year == 2021:
        return row['report_date'].month
    elif row['report_date'].year == 2022:
        return row['report_date'].month + 12
    elif row['report_date'].year == 2023:
        return row['report_date'].month + 24

In [ ]:
train['month'] = train.apply(date_maker, axis=1)

In [ ]:
test['month'] = test.apply(date_maker, axis=1)

In [ ]:
train = train.merge(train.groupby('client_id')['month'].agg('min').reset_index(), \
                    how='left', on='client_id', suffixes=('', '_min'))

In [ ]:
test = test.merge(test.groupby('client_id')['month'].agg('min').reset_index(), \
                    how='left', on='client_id', suffixes=('', '_min'))

In [ ]:
train['month_from_first'] = train['month'] - train['month_min'] + 1

In [ ]:
test['month_from_first'] = test['month'] - test['month_min'] + 1

In [ ]:
train = train.drop(columns=['month', 'month_min'])

In [ ]:
test = test.drop(columns=['month', 'month_min'])

In [ ]:
train.head()

In [ ]:
features_train = train.drop(columns=['report_date', 'client_id', 'target'])

In [ ]:
target_train = train['target']

In [ ]:
features_train, target_train = SMOTE().fit_resample(features_train, target_train)

In [ ]:
features_test = test.drop(columns=['report_date', 'client_id', 'target'])

In [ ]:
target_test = test['target']

In [ ]:
params = {
    'loss_function': 'Logloss',
    'iterations': 300,
    'custom_loss': ['AUC', 'F1'],
    'random_seed': RS,
    'learning_rate': 0.2
}

In [ ]:
cv_data = cv(
    params=params,
    pool=Pool(features_train, label=target_train),
    fold_count=5, # Разбивка выборки на 5 кусочков
    shuffle=True, # Перемешаем наши данные
    partition_random_seed=RS,
    plot=True,
    stratified=True, 
    verbose=False
)

In [ ]:
cv_data

In [ ]:
feature_month_auc_valid = cv_data['test-AUC-mean'].tail(1)

In [ ]:
feature_month_f1_valid = cv_data['test-F1-mean'].tail(1)

In [ ]:
feature_month_auc_valid

In [ ]:
feature_month_f1_valid

### Валидация модели

In [ ]:
model = CatBoostClassifier(loss_function='Logloss',
    iterations=300,
    custom_loss=['AUC', 'F1'],
    random_seed= RS,
    learning_rate= 0.2,
    verbose=False)

In [ ]:
model.fit(features_train, target_train)

In [ ]:
pred = model.predict(features_test)

In [ ]:
feature_month_auc_test = roc_auc_score(target_test, pred)

In [ ]:
feature_month_auc_test

In [ ]:
feature_month_f1_test = f1_score(target_test, pred)

In [ ]:
feature_month_f1_test

In [ ]:
cm = confusion_matrix(target_test, pred, normalize='true')
ConfusionMatrixDisplay(confusion_matrix=cm).plot()
plt.show()

## Бейслайн модель с новыми фичами (месяц года)

In [ ]:
data_f

In [ ]:
train, test = train_test_split(data_f, test_size=0.3, random_state=RS, stratify=data['target'])

In [ ]:
def month_maker(row):
        return row['report_date'].month

In [ ]:
train['month'] = train.apply(month_maker, axis=1)

In [ ]:
test['month'] = test.apply(month_maker, axis=1)

In [ ]:
train.head()

In [ ]:
train = pd.get_dummies(train, columns=['month'], drop_first=False, dtype='int')

In [ ]:
train.head()

In [ ]:
test = pd.get_dummies(test, columns=['month'], drop_first=False, dtype='int')

In [ ]:
features_train = train.drop(columns=['report_date', 'client_id', 'target'])

In [ ]:
target_train = train['target']

In [ ]:
features_train, target_train = SMOTE().fit_resample(features_train, target_train)

In [ ]:
features_test = test.drop(columns=['report_date', 'client_id', 'target'])

In [ ]:
target_test = test['target']

In [ ]:
params = {
    'loss_function': 'Logloss',
    'iterations': 300,
    'custom_loss': ['AUC', 'F1'],
    'random_seed': RS,
    'learning_rate': 0.2
}

In [ ]:
cv_data = cv(
    params=params,
    pool=Pool(features_train, label=target_train),
    fold_count=5, # Разбивка выборки на 5 кусочков
    shuffle=True, # Перемешаем наши данные
    partition_random_seed=RS,
    plot=True,
    stratified=True, 
    verbose=False
)

In [ ]:
cv_data

In [ ]:
feature_season_auc_valid = cv_data['test-AUC-mean'].tail(1)

In [ ]:
feature_season_f1_valid = cv_data['test-F1-mean'].tail(1)

In [ ]:
feature_season_auc_valid

In [ ]:
feature_season_f1_valid

### Валидация модели

In [ ]:
model = CatBoostClassifier(loss_function='Logloss',
    iterations=300,
    custom_loss=['AUC', 'F1'],
    random_seed= RS,
    learning_rate= 0.2,
    verbose=False)

In [ ]:
model.fit(features_train, target_train)

In [ ]:
pred = model.predict(features_test)

In [ ]:
feature_season_auc_test = roc_auc_score(target_test, pred)

In [ ]:
feature_season_auc_test

In [ ]:
feature_season_f1_test = f1_score(target_test, pred)

In [ ]:
feature_season_f1_test

In [ ]:
cm = confusion_matrix(target_test, pred, normalize='true')
ConfusionMatrixDisplay(confusion_matrix=cm).plot()
plt.show()

## Модель на основе нейронной сети

In [ ]:
features = data.drop(columns=['target', 'report_date'])

In [ ]:
target = data['target']

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, stratify=target,
                                                                            random_state=RS)

In [ ]:
features_train, target_train = SMOTE().fit_resample(features_train, target_train)

In [ ]:
def baseline_nn(features_train):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(units=features_train.shape[1], input_dim=features_train.shape[1], 
                                 activation='sigmoid'))
    model.add(keras.layers.Dense(units=1, input_dim=features_train.shape[1], 
                                 activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['acc', AUC()])
    
    return model

In [ ]:
model = baseline_nn(features_train)

In [ ]:
model.fit(features_train, target_train, epochs=10, verbose=2,
          validation_data=(features_test, target_test))